In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('../../../datasets/processed_data/combined_features/IPCALAB.BO.csv')

In [3]:
df.columns

Index(['date', 'open', 'close', 'adj close', 'volume', 'high', 'low',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'treasury_yeild_10_years_percent_change', 'usdx-index_percent_change',
       '^nsei_percent_change', '^bsesn_percent_change', '^gspc_percent_change',
       'hsi_percent_change', 'sha_percent_change', '^sti_percent_change',
       'yesterday_close', 'ln_target'],
      dtype='object', length=635)

In [4]:
def create_custom_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates the custom target price, which is computed as ln(high/yesterday_close)
    """    
    # make a copy of the dataframe so as not to change the original dataframe
    data_df = df.copy()
    
    # create yesterday_close feature as
    data_df['yesterday_close'] = data_df['close'].shift(1)
    
    # create custom target price to predict, computing  ln(high/yesterday_close)
    data_df['ln_target'] = np.log(data_df['high'] / data_df['yesterday_close'])
    
    # as yesterday close would not be available for first day, 
    # we would not have custom target price for that day, which needs to be excluded 
    return(data_df.iloc[1:, ])

In [5]:
df = create_custom_target(df)

In [6]:
df['yesterday_close']

1        59.625000
2        60.000000
3        58.235001
4        57.294998
5        56.040001
           ...    
3684    689.049988
3685    684.150024
3686    682.250000
3687    683.599976
3688    687.250000
Name: yesterday_close, Length: 3688, dtype: float64

In [7]:
df['ln_target'] 

1       0.029335
2       0.021435
3       0.023169
4       0.003572
5       0.107500
          ...   
3684    0.002030
3685    0.005175
3686    0.004752
3687    0.014450
3688    0.028333
Name: ln_target, Length: 3688, dtype: float64

In [8]:
# Drop cols with NaN values
df = df.dropna(axis=1, how='all')
df.fillna(method='ffill', inplace=True)

In [9]:
combined_date_df = df['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)

In [10]:
df_without_date = df.drop(columns=["date"])

In [11]:
df_without_date.dropna(inplace=True)

In [12]:
def timeseries_to_supervise(df, window_size, target):    
    X = []
    y = []
    indx = []
    no_records = len(df)
    #     
    for i in range(window_size, no_records):
        X.append(df.iloc[i-window_size:i].drop(target, axis=1).values.flatten())  # Collect past records as a sequence
        y.append(df.iloc[i][target])  # Next record as target variable
        indx.append(np.arange(i-window_size, i))

    X = pd.DataFrame(X)
    y = pd.Series(y)
    return(X, y, indx)

In [13]:
window_size = 10

In [14]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# Identify the indices of the specified sentiment columns in the original dataset
sentiment_columns = ['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound']
column_indices = [train_data.columns.get_loc(col) for col in sentiment_columns if col in train_data.columns]

column_indices

[616, 617, 618, 619, 620, 621]

In [15]:
# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [16]:
# Use SelectKBest to select the top 44 features
selector = SelectKBest(score_func=f_regression, k=44)
X_train_44 = selector.fit_transform(X_train, y_train)
X_test_44 = selector.transform(X_test)

# Determine if the specified columns are among the top 44 features
selected_indices = selector.get_support(indices=True)
missing_indices = [idx for idx in column_indices if idx not in selected_indices]

# Add the specified columns if they are not among the top 44 features
for idx in missing_indices:
    extracted_column_train = X_train.iloc[:, idx].values.reshape(-1, 1)
    extracted_column_test = X_test.iloc[:, idx].values.reshape(-1, 1)
    X_train_44 = np.hstack([X_train_44, extracted_column_train])
    X_test_44 = np.hstack([X_test_44, extracted_column_test])

# If some of the specified columns were already in the top 44, 
# select additional top features to make the total count 50
remaining_indices = [i for i in range(X_train.shape[1]) if i not in selected_indices and i not in missing_indices]
num_additional_features_needed = 50 - X_train_44.shape[1]

if num_additional_features_needed > 0:
    additional_selector = SelectKBest(score_func=f_regression, k=num_additional_features_needed)
    additional_selector.fit(X_train.iloc[:, remaining_indices], y_train)
    X_train_additional = additional_selector.transform(X_train.iloc[:, remaining_indices])
    X_test_additional = additional_selector.transform(X_test.iloc[:, remaining_indices])

    X_train_44 = np.hstack([X_train_44, X_train_additional])
    X_test_44 = np.hstack([X_test_44, X_test_additional])

X_train_50 = X_train_44
X_test_50 = X_test_44

In [17]:
linear_reg = LinearRegression()

param_grid = {
    'fit_intercept': [True, False],         
}

grid_search = GridSearchCV(linear_reg, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_50, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)

model_50 = grid_search.best_estimator_
model_50.fit(X_train_50, y_train)

Best parameters:  {'fit_intercept': False}
Best cross-validation score:  0.19532697153266296


LinearRegression(fit_intercept=False)

In [18]:
def convert_custom_target_to_actual_for_supervise(df: pd.DataFrame, window: int, y: "pd.Series[int]") -> "pd.Series[int]":
    """
    this module converts custom target - ln(high/yesterday_close) to actual high price again for timeseries converted data using rolling         window of size 10
    """
    data_df = df.copy()
    
    # exclude first 10 rows of train/test data, as while us
    
    y = np.exp(y) * data_df.loc[data_df.index[window:], 'yesterday_close'].reset_index(drop=True)
    return(y)    

In [19]:
def evaluate_model(model, window, dev_data, dev_date, X_test, y_test):
    
    # do target prediction using the provide model
    y_pred = model.predict(X_test)

    # convert back to original value, before computing mape            
    y_test = convert_custom_target_to_actual_for_supervise(dev_data, window, y_test)
    y_pred = convert_custom_target_to_actual_for_supervise(dev_data, window, y_pred)

    dev_dates = dev_date[window:].reset_index(drop=True)
    predictions_df = pd.DataFrame({'date': dev_dates, 'y_test': y_test, 'y_pred': y_pred})

    # compute regression metric - mape 
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # compute rmse metric
    rmse = mean_squared_error(y_test, y_pred, squared=False)        
    return(predictions_df, mape, rmse)

In [20]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, test_data, test_date, X_test_50, y_test)


In [26]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-06-23  837.500000  842.961857
 1    2020-06-24  836.750000  845.728556
 2    2020-06-25  842.500000  842.349669
 3    2020-06-26  854.575012  847.961796
 4    2020-06-29  853.799988  856.607605
 ..          ...         ...         ...
 723  2023-05-24  690.450012  701.908607
 724  2023-05-25  687.700012  693.954612
 725  2023-05-26  685.500000  683.552557
 726  2023-05-30  693.549988  687.703219
 727  2023-05-31  707.000000  689.744093
 
 [728 rows x 3 columns],
 0.016917475465404102,
 23.20665136351524)

In [27]:
print(f"MAPE for model with top 50 features: {mape*100:.2f}%")

MAPE for model with top 50 features: 1.69%


In [28]:
predictions_df.to_csv("lr_IPCALAB.csv", index=False)

In [29]:
feature_names = df.columns.tolist()

# Get coefficients from the model
coefficients = model_50.coef_

# Map coefficients to corresponding feature names
feature_coefficient_mapping = dict(zip(feature_names, coefficients))

# Sort by absolute coefficient values (magnitude)
sorted_features = sorted(feature_coefficient_mapping.items(), key=lambda x: abs(x[1]), reverse=True)

# Display top 10
for feature, coef in sorted_features[:10]:
    print(f"{feature}: {coef}")

trend_ema_slow: -0.024901194955417212
trend_vortex_ind_neg: -0.017470775433110285
volatility_ui: -0.014694139393886892
volatility_dcm: 0.007542665304624795
trend_ema_fast: 0.005777296767652163
volatility_atr: 0.0036751962136713353
trend_macd_signal: 0.002330681873434122
trend_vortex_ind_diff: 0.0023129613062258018
adj close: 0.0018295402813005535
volatility_dch: -0.0017105882311713575


In [30]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

Repeat the process for the top 50 features without sentiment of IPCALAB.BO

In [32]:
df = pd.read_csv('../../../datasets/processed_data/combined_features/IPCALAB.BO.csv')

In [33]:
df_without_sentiment = df.drop(columns=['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound'])

In [34]:
df_without_sentiment = create_custom_target(df_without_sentiment)

In [35]:
# Drop cols with NaN values
df_without_sentiment = df_without_sentiment.dropna(axis=1, how='all')
df_without_sentiment.fillna(method='ffill', inplace=True)

In [36]:
combined_date_df = df_without_sentiment['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)

In [37]:
df_without_date = df_without_sentiment.drop(columns=["date"])

In [38]:
df_without_date.dropna(inplace=True)

In [39]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [40]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [41]:
linear_reg = LinearRegression()

param_grid = {
    'fit_intercept': [True, False],         
}

grid_search = GridSearchCV(linear_reg, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_50, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)

model_50 = grid_search.best_estimator_
model_50.fit(X_train_50, y_train)

Best parameters:  {'fit_intercept': True}
Best cross-validation score:  0.4293450907956207


LinearRegression()

In [42]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, test_data, test_date, X_test_50, y_test)

In [43]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-06-23  837.500000  841.201533
 1    2020-06-24  836.750000  853.109688
 2    2020-06-25  842.500000  836.297250
 3    2020-06-26  854.575012  847.757228
 4    2020-06-29  853.799988  858.100084
 ..          ...         ...         ...
 723  2023-05-24  690.450012  699.332076
 724  2023-05-25  687.700012  691.729586
 725  2023-05-26  685.500000  683.421530
 726  2023-05-30  693.549988  686.099509
 727  2023-05-31  707.000000  688.809848
 
 [728 rows x 3 columns],
 0.017210257173806566,
 24.118885904119118)

In [38]:
print(f"MAPE for model with top 50 features without sentiment features: {mape*100:.2f}%")

MAPE for model with top 50 features without sentiment features: 1.72%


In [44]:
predictions_df.to_csv("lr_IPCALAB_without_sentiment.csv", index=False)

In [45]:
feature_names = df.columns.tolist()

# Get coefficients from the model
coefficients = model_50.coef_

# Map coefficients to corresponding feature names
feature_coefficient_mapping = dict(zip(feature_names, coefficients))

# Sort by absolute coefficient values (magnitude)
sorted_features = sorted(feature_coefficient_mapping.items(), key=lambda x: abs(x[1]), reverse=True)

# Display top 10
for feature, coef in sorted_features[:10]:
    print(f"{feature}: {coef}")

trend_ema_fast: -0.014249586100161915
trend_macd_signal: 0.00920681087552564
trend_mass_index: 0.0050011127919399245
volatility_kcc: 0.0036255739837416224
trend_sma_slow: 0.0036072843195179417
volatility_dch: -0.002150503097074147
trend_vortex_ind_pos: 0.002128485489495828
trend_macd: -0.002105830242282236
volume_obv: 0.0017077954204586558
volatility_bbp: -0.0016029026180189488


In [46]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)